# 🚀 Web Server – Control de LED y Sensor Virtual

> Controla un LED y genera valores dinámicos desde tu navegador usando **MicroPython** y un **ESP32**.

---

### 📄 Información del Proyecto
- **👤 Autor:** Raze
- **📅 Fecha:** Noviembre 2025  
- **🧰 Versión:** 1.0  
- **🖥️ Plataforma:** ESP32 DevKit v1  
- **💬 Lenguaje:** MicroPython  
- **📁 Repositorio:** [GitHub/ESP32_WebServer](#)

---

### 🧠 Objetivo
Desarrollar un servidor web embebido en el ESP32 para:
- Controlar el LED integrado desde una página web.  
- Mostrar una lectura de temperatura.  
- Generar y visualizar un número aleatorio.

---

### Librerías necesarias
- network: para conectarse a una red WiFi
- socket: para crear el servidor web
- random: para generar valores aleatorios
- time: para pausas y sincronización
- machine: para controlar los pines físicos del ESP32
- sys: para gestionar errores críticos

In [ ]:
import network
import socket
import random
from time import sleep
from machine import Pin
import sys

## Configuración de red WiFi
Cambia el SSID y la contraseña antes de ejecdeutar el código.

In [ ]:
ssid = 'raze'
password = 'armandocode'

## Configuración del LED
Se utiliza el pin GPIO 2 (LED integrado en la mayoría de ESP32).

In [ ]:
led = Pin(2, Pin.OUT)

## Generación de la página web

In [ ]:
def webpage(state, temperature, random_value):
    """Genera el HTML dinámico que se muestra al usuario."""
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Servidor ESP32</title>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <style>
            body {{ font-family: Arial, sans-serif; background-color: #f7f7f7; text-align: center; }}
            h1 {{ color: #007bff; }}
            .button {{
                display: inline-block;
                width: 200px;
                padding: 10px;
                margin: 10px;
                border-radius: 5px;
                text-decoration: none;
                color: white;
            }}
            .on {{ background-color: #4CAF50; }}
            .off {{ background-color: #f44336; }}
            .rand {{ background-color: #c0392b; }}
        </style>
    </head>
    <body>
        <h1>Control de LED</h1>
        <a class="button on" href="/lighton?">Encender luz LED</a>
        <a class="button off" href="/lightoff?">Apagar luz LED</a>
        <p>El LED está: <strong>{state}</strong></p>
        
        <h2>Obtener nuevo valor aleatorio</h2>
        <a class="button rand" href="/random?">Obtener aleatorio</a>
        <p>Valor obtenido: <strong>{random_value}</strong></p>

        <h2>Resumen final</h2>
        <p>El LED está: <strong>{state}</strong></p>
        <p>La temperatura es: <strong>{temperature:.2f}</strong> °C</p>
        <p>Valor obtenido: <strong>{random_value}</strong></p>
    </body>
    </html>
    """
    return html

## Conexión a la red WiFi

In [ ]:
def connect():
wlan = network.WLAN(network.STA_IF)
wlan.active(True)
wlan.connect(ssid, password)


print('Conectando a la red...')
for _ in range(15):
if wlan.isconnected():
break
print('.', end='')
sleep(1)


if wlan.isconnected():
print('\nConectado a WiFi')
print('IP:', wlan.ifconfig()[0])
return wlan.ifconfig()[0]
else:
print('\nError: no se pudo conectar')
sys.exit()

## Creación del socket servidor

In [ ]:
def open_socket(ip):
address = (ip, 80)
connection = socket.socket()
connection.bind(address)
connection.listen(1)
print("Servidor listo en:", ip)
return connection

## 8. Bucle principal del servidor

In [ ]:
def serve(connection):
state = 'OFF'
led.value(0)
random_value = 0
temperature = random.uniform(25.0, 35.0)


while True:
client, addr = connection.accept()
print('Cliente conectado desde', addr)
request = client.recv(1024)
request = str(request)


if '/lighton?' in request:
led.value(1)
state = 'ON'
elif '/lightoff?' in request:
led.value(0)
state = 'OFF'
elif '/random?' in request:
random_value = random.randint(0, 100)
temperature = random.uniform(25.0, 35.0)


html = webpage(state, temperature, random_value)
client.send('HTTP/1.0 200 OK\r\nContent-type: text/html\r\n\r\n')
client.send(html)
client.close()

## Ejecución del programa

In [ ]:
try:
    ip = connect()
    connection = open_socket(ip)
    serve(connection)
except Exception as e:
    print("Error del servidor:", e)
    sys.exit()

## Conclusión
- Este proyecto demuestra el uso del ESP32 como un servidor web básico con MicroPython.
- Muestra cómo interactuar con hardware (LED) desde una página web y cómo generar
- datos dinámicos para visualización en tiempo real.

![Vista previa del servidor ESP32](images/web_server.png)